In [65]:
# importing the model

import joblib
model = joblib.load('XGBoost_final.pkl')

In [60]:
# loading the test dataset

import pandas as pd
test_Df = pd.read_excel('TestDatasetExample (1).xls') # change this to the required file name/path.

In [61]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# preprocessing and null value removal.

test_Df=test_Df.replace(999, None)

#imputation using mode
for col in test_Df.columns:
    test_Df[col].fillna(test_Df[col].mode()[0], inplace=True)

# only column 11 onwards taken for pca
test_Df_forPCA = test_Df.iloc[:,11:]

# Standardize the features
scaler = StandardScaler()
test_Df_forPCA = scaler.fit_transform(test_Df_forPCA)

# Perform PCA
pca = PCA(n_components=6)  # Reduce to 6 principal components
test_Df_afterPCA = pca.fit_transform(test_Df_forPCA)

test_Df_afterPCA = pd.DataFrame(test_Df_afterPCA)
test_Df = test_Df.iloc[:,0:11].merge(test_Df_afterPCA, left_index = True, right_index = True, how = 'right')

new_column_names = {0: 'COMP0', 1: 'COMP1', 2: 'COMP2',3: 'COMP3',4:'COMP4',5:'COMP5',}
test_Df = test_Df.rename(columns=new_column_names)

# scaling all the values
test_Df[['Age']] = StandardScaler().fit_transform(test_Df[['Age']])

#drop proliferation as its not needed.
test_Df = test_Df.drop(columns=['Proliferation'])

In [62]:
# Making predictions on data
test_predictions = model.predict(test_Df.drop(columns=['ID']))

In [63]:
test_predictions = pd.DataFrame(test_predictions, columns=['pCR(result)'])

In [64]:
to_excel_Df = pd.DataFrame(test_Df['ID'])
#data after handling missing values
to_excel_Df = pd.concat((to_excel_Df, test_predictions), axis=1)
to_excel_Df.to_excel('FinalTestPCR.xlsx', index=False, header=True)

                        